In [9]:
import pandas as pd

datafile1 = "financial_compliance_source.csv"
datafile2 = "financial_compliance_target.csv"
targetTable = pd.read_csv(datafile2)
inputTable = pd.read_csv(datafile1)
targetTable

,year,firm_name,total_audit_engagement,high_risk_cases_count,compliance_violations_count,fraud_cases_count,industry_affected,revenue_impact_millions,ai_used,employee_workload_percent,audit_impact_score,client_satisfaction_score,overall_score
0,2024,Deloitte,1581,247,181,72,Retail,221.42,0,56,5.6,7.9,6.75
1,2025,Deloitte,760,406,65,69,Finance,140.29,1,40,5.1,7.5,6.30
2,2023,PwC,2238,101,184,79,Retail,216.84,1,65,7.0,7.1,7.05
3,2024,Ernst & Young,1760,415,183,11,Healthcare,291.00,1,53,8.2,8.4,8.30
4,2024,PwC,4103,423,179,30,Healthcare,225.42,1,41,6.7,5.0,5.85


In [11]:
inputTable

,Year,Firm_Name,Total_Audit_Engagements,High_Risk_Cases,Compliance_Violations,Fraud_Cases_Detected,Industry_Affected,Total_Revenue_Impact,AI_Used_for_Auditing,Employee_Workload,Audit_Effectiveness_Score,Client_Satisfaction_Score
0,2020.0,NaN,2829.0,5100.0,NaN,39.0,Healthcare,114.24,NaN,57.0,5.8,8.4
1,NaN,Deloitte,NaN,185.0,30.0,6000.0,NaN,156.98,Yes,NaN,5.3,670.0
2,NaN,NaN,243800.0,NaN,NaN,NaN,NaN,131.83,NaN,76.0,NaN,NaN
3,202100.0,NaN,264600.0,397.0,55.0,97.0,NaN,22911.00,No,6000.0,5.1,860.0
4,NaN,PwC,2680.0,216.0,99.0,NaN,Healthcare,48.00,No,NaN,9.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
111,2021.0,KPMG,2503.0,NaN,NaN,62.0,NaN,228.15,NaN,69.0,7.4,NaN
112,2025.0,PwC,1771.0,NaN,NaN,NaN,Tech,48564.00,NaN,68.0,NaN,NaN
113,NaN,Deloitte,797.0,133.0,46.0,27.0,Finance,NaN,No,77.0,6.3,8.6
114,2022.0,KPMG,NaN,NaN,NaN,69.0,Healthcare,89.79,Yes,NaN,7.6,5.6
